<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wandb

**dependencies**

In [3]:
# used snniped from https://github.com/L1aoXingyu/pytorch-beginner/
import torch
import wandb
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Process

**initial**

In [ ]:
(xtrain,ytrain), (xtest,ytest) = mnist.load_data()
num_epochs=100      #
batch_size = 64     #
image_size=784      #
hidden_size=64      #
lv_size = 64        # Latent Variable 
learning_rate=1e-4  #
cret = nn.MSELoss() # criterion

**build model**

In [5]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, hidden_size),   #nn.ReLU(True), nn.Linear(image_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size), nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**model setting**

In [6]:
model = autoencoder()
#wandb.watch(model)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])

def images_row(dis_images,title,add_to_index=0, images_in_row=5):
  if( len(np.shape(dis_images)) == 2):
    dis_images=dis_images[0:images_in_row,:]
  else:
    dis_images=dis_images[0:images_in_row,0,:,:]
  for i in range(len(dis_images)):
    ax = plt.subplot(30, images_in_row, i+add_to_index + 1)
    plt.imshow(dis_images[i].reshape(28, 28))
    plt.title( title)
    plt.gray()

def visual_epoch(epoch_num,model,dataloader):
  for data in dataloader:
      input_imgs = data
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      output_imgs = model(imgs)
      images_row(input_imgs,"org ",5*epoch_num)
      images_row(output_imgs.detach().numpy(),"rec ",5*(epoch_num+1))

**train model by digit**

In [7]:
def train_by_digit(by_digit,model):
  wandb.init()
  print("*****\nstart traning Model for digit " +str(by_digit) +"\n")
  dataloader = DataLoader(DigitDataSet(xtrain[ytrain==by_digit]), batch_size=batch_size,shuffle=True, num_workers=4)
  visual_counter=0
  for epoch in range(num_epochs):
    run=0
    if epoch%10==0:
      c=1
    for data in dataloader:
      input_imgs = data
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if run:
        run=0
        im=data[0,0,:,:].reshape(28,28)
        pred=model(imgs).detach().numpy()[0,:].reshape(28,28)
        wandb.log({"img": [wandb.Image(pred, caption="preidciton"),wandb.Image(im, caption="original")]})
    #if epoch%3==0 :
    #  visual_epoch(visual_counter,model,dataloader)
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    wandb.log({"loss": loss.data})

  model_name='./ae_'+str(by_digit)+'.pth'
  torch.save(model.state_dict(),model_name )
  wandb.save(model_name)
  print("\nfinish traning Model Number " +str(by_digit) +"\n")
  print("*****\n")

for by_digit in range(10):
  train_by_digit(by_digit,model)
  

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


*****
start traning Model for digit 0

epoch [1/100], loss:0.5953
epoch [2/100], loss:0.2641
epoch [3/100], loss:0.2541
epoch [4/100], loss:0.2364
epoch [5/100], loss:0.2595
epoch [6/100], loss:0.2499
epoch [7/100], loss:0.2434
epoch [8/100], loss:0.2198
epoch [9/100], loss:0.2276
epoch [10/100], loss:0.1962
epoch [11/100], loss:0.1953
epoch [12/100], loss:0.2117
epoch [13/100], loss:0.1841
epoch [14/100], loss:0.1932
epoch [15/100], loss:0.1886
epoch [16/100], loss:0.1676
epoch [17/100], loss:0.1628
epoch [18/100], loss:0.1536
epoch [19/100], loss:0.1659
epoch [20/100], loss:0.1502
epoch [21/100], loss:0.1574
epoch [22/100], loss:0.1367
epoch [23/100], loss:0.1226
epoch [24/100], loss:0.1410
epoch [25/100], loss:0.1305
epoch [26/100], loss:0.1365
epoch [27/100], loss:0.1223
epoch [28/100], loss:0.1468
epoch [29/100], loss:0.1329
epoch [30/100], loss:0.1197
epoch [31/100], loss:0.1404
epoch [32/100], loss:0.1197
epoch [33/100], loss:0.1218
epoch [34/100], loss:0.1135
epoch [35/100], lo

...

*****
start traning Model for digit 7

epoch [1/100], loss:0.0956
epoch [2/100], loss:0.0774
epoch [3/100], loss:0.0696
epoch [4/100], loss:0.0650
epoch [5/100], loss:0.0688
epoch [6/100], loss:0.0714
epoch [7/100], loss:0.0650
epoch [8/100], loss:0.0669
epoch [9/100], loss:0.0516
epoch [10/100], loss:0.0550
epoch [11/100], loss:0.0525
epoch [12/100], loss:0.0611
epoch [13/100], loss:0.0579
epoch [14/100], loss:0.0537
epoch [15/100], loss:0.0565
epoch [16/100], loss:0.0488
epoch [17/100], loss:0.0445
epoch [18/100], loss:0.0459
epoch [19/100], loss:0.0560
epoch [20/100], loss:0.0582
epoch [21/100], loss:0.0460
epoch [22/100], loss:0.0444
epoch [23/100], loss:0.0445
epoch [24/100], loss:0.0488
epoch [25/100], loss:0.0453
epoch [26/100], loss:0.0522
epoch [27/100], loss:0.0516
epoch [28/100], loss:0.0396
epoch [29/100], loss:0.0401
epoch [30/100], loss:0.0457
epoch [31/100], loss:0.0394
epoch [32/100], loss:0.0538
epoch [33/100], loss:0.0504
epoch [34/100], loss:0.0442
epoch [35/100], lo

*****
start traning Model for digit 8

epoch [1/100], loss:0.1232
epoch [2/100], loss:0.1137
epoch [3/100], loss:0.1198
epoch [4/100], loss:0.1137
epoch [5/100], loss:0.1118
epoch [6/100], loss:0.0960
epoch [7/100], loss:0.0936
epoch [8/100], loss:0.0954
epoch [9/100], loss:0.1059
epoch [10/100], loss:0.1066
epoch [11/100], loss:0.1004
epoch [12/100], loss:0.0874
epoch [13/100], loss:0.0944
epoch [14/100], loss:0.0803
epoch [15/100], loss:0.0799
epoch [16/100], loss:0.0930
epoch [17/100], loss:0.0744
epoch [18/100], loss:0.0796
epoch [19/100], loss:0.0844
epoch [20/100], loss:0.0835
epoch [21/100], loss:0.0824
epoch [22/100], loss:0.0844
epoch [23/100], loss:0.0829
epoch [24/100], loss:0.0909
epoch [25/100], loss:0.0799
epoch [26/100], loss:0.0822
epoch [27/100], loss:0.0640
epoch [28/100], loss:0.0794
epoch [29/100], loss:0.0717
epoch [30/100], loss:0.0803
epoch [31/100], loss:0.0807
epoch [32/100], loss:0.0841
epoch [33/100], loss:0.0739
epoch [34/100], loss:0.0704
epoch [35/100], lo

*****
start traning Model for digit 9

epoch [1/100], loss:0.0724
epoch [2/100], loss:0.0705
epoch [3/100], loss:0.0627
epoch [4/100], loss:0.0615
epoch [5/100], loss:0.0563
epoch [6/100], loss:0.0536
epoch [7/100], loss:0.0536
epoch [8/100], loss:0.0504
epoch [9/100], loss:0.0554
epoch [10/100], loss:0.0567
epoch [11/100], loss:0.0472
epoch [12/100], loss:0.0568
epoch [13/100], loss:0.0500
epoch [14/100], loss:0.0519
epoch [15/100], loss:0.0482
epoch [16/100], loss:0.0490
epoch [17/100], loss:0.0406
epoch [18/100], loss:0.0449
epoch [19/100], loss:0.0419
epoch [20/100], loss:0.0477
epoch [21/100], loss:0.0446
epoch [22/100], loss:0.0461
epoch [23/100], loss:0.0456
epoch [24/100], loss:0.0413
epoch [25/100], loss:0.0435
epoch [26/100], loss:0.0393
epoch [27/100], loss:0.0448
epoch [28/100], loss:0.0447
epoch [29/100], loss:0.0469
epoch [30/100], loss:0.0480
epoch [31/100], loss:0.0386
epoch [32/100], loss:0.0378
epoch [33/100], loss:0.0385
epoch [34/100], loss:0.0403
epoch [35/100], lo

requests_with_retry encountered retryable exception: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')). args: ('https://api.wandb.ai/files/aviax1/uncategorized/218phxcm/file_stream',), kwargs: {'json': {'files': {'output.log': {'offset': 367, 'content': ['2020-07-23T00:15:33.791677 epoch [41/100], loss:0.0403\n', '2020-07-23T00:15:37.303410 epoch [42/100], loss:0.0365\n', '2020-07-23T00:15:40.753073 epoch [43/100], loss:0.0389\n', '2020-07-23T00:15:44.180219 epoch [44/100], loss:0.0455\n', '2020-07-23T00:15:47.564930 epoch [45/100], loss:0.0393\n', '2020-07-23T00:15:50.960753 epoch [46/100], loss:0.0379\n', '2020-07-23T00:15:54.946023 epoch [47/100], loss:0.0349\n', '2020-07-23T00:15:58.404473 epoch [48/100], loss:0.0484\n', '2020-07-23T00:16:01.869667 epoch [49/100], loss:0.0407\n']}, 'wandb-events.jsonl': {'offset': 41, 'content': ['{"system.cpu": 62.9, "system.memory": 8.49, "system.disk": 30.3, "system.proc.memory.availableMB": 11916.16, "system.proc.me

epoch [50/100], loss:0.0346
epoch [51/100], loss:0.0411
epoch [52/100], loss:0.0438
epoch [53/100], loss:0.0383
epoch [54/100], loss:0.0349
epoch [55/100], loss:0.0336
epoch [56/100], loss:0.0366
epoch [57/100], loss:0.0378
epoch [58/100], loss:0.0355
epoch [59/100], loss:0.0378
epoch [60/100], loss:0.0353
epoch [61/100], loss:0.0360
epoch [62/100], loss:0.0399
epoch [63/100], loss:0.0358
epoch [64/100], loss:0.0319
epoch [65/100], loss:0.0374
epoch [66/100], loss:0.0406
epoch [67/100], loss:0.0336
epoch [68/100], loss:0.0305
epoch [69/100], loss:0.0430
epoch [70/100], loss:0.0340
epoch [71/100], loss:0.0416
epoch [72/100], loss:0.0332
epoch [73/100], loss:0.0334
epoch [74/100], loss:0.0349
epoch [75/100], loss:0.0375
epoch [76/100], loss:0.0377
epoch [77/100], loss:0.0406
epoch [78/100], loss:0.0392
epoch [79/100], loss:0.0380
epoch [80/100], loss:0.0329
epoch [81/100], loss:0.0332
epoch [82/100], loss:0.0386
epoch [83/100], loss:0.0378
epoch [84/100], loss:0.0363
epoch [85/100], loss